## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Anadyr,RU,64.7500,177.4833,-35.52,73,64,6.80,broken clouds
1,1,Mahebourg,MU,-20.4081,57.7000,76.24,79,12,1.59,few clouds
2,2,Zatoka,UA,46.0725,30.4654,37.94,71,0,4.03,clear sky
3,3,Butaritari,KI,3.0707,172.7902,81.46,78,79,15.26,broken clouds
4,4,Bluff,NZ,-46.6000,168.3333,57.79,67,21,14.03,few clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mahebourg,MU,-20.4081,57.7000,76.24,79,12,1.59,few clouds
3,3,Butaritari,KI,3.0707,172.7902,81.46,78,79,15.26,broken clouds
12,12,Hithadhoo,MV,-0.6000,73.0833,81.48,70,100,8.16,overcast clouds
14,14,Rikitea,PF,-23.1203,-134.9692,79.03,73,4,16.28,clear sky
18,18,Cidreira,BR,-30.1811,-50.2056,77.47,76,0,19.04,clear sky
20,20,Kapaa,US,22.0752,-159.3190,76.19,69,0,5.75,clear sky
21,21,Luderitz,NaN,-26.6481,15.1594,78.48,61,100,16.15,overcast clouds
22,22,Mattru,SL,7.6244,-11.8332,78.91,76,7,6.78,clear sky
33,33,Kieta,PG,-6.2167,155.6333,78.03,86,100,1.68,light rain
34,34,Chuy,UY,-33.6971,-53.4616,79.32,67,94,15.93,overcast clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                196
City                   196
Country                194
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,76.24,few clouds,-20.4081,57.7000,
3,Butaritari,KI,81.46,broken clouds,3.0707,172.7902,
12,Hithadhoo,MV,81.48,overcast clouds,-0.6000,73.0833,
14,Rikitea,PF,79.03,clear sky,-23.1203,-134.9692,
18,Cidreira,BR,77.47,clear sky,-30.1811,-50.2056,
20,Kapaa,US,76.19,clear sky,22.0752,-159.3190,
21,Luderitz,NaN,78.48,overcast clouds,-26.6481,15.1594,
22,Mattru,SL,78.91,clear sky,7.6244,-11.8332,
33,Kieta,PG,78.03,light rain,-6.2167,155.6333,
34,Chuy,UY,79.32,overcast clouds,-33.6971,-53.4616,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == ''].index, inplace = False)
clean_hotel_df.shape

(180, 7)

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))